### Comparar Score del modelo con:
- Modelo sin upsampling
- Modelo con upsampling tradicional
- Modelo con datos sinteticos sobre toda la muestra
- Modelo con datos sintéticod sobre la clase menor

In [3]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns

In [4]:
df = pd.read_parquet('../data/processed/train.parquet')

In [5]:
df.dtypes

term                      int32
int_rate                float32
sub_grade                object
emp_length                int32
home_ownership           object
annual_inc              float64
verification_status       int64
purpose                  object
addr_state               object
dti                     float64
delinq_2yrs             float64
earliest_cr_line         object
fico_range_low          float64
open_acc                float64
pub_rec                 float64
revol_bal               float64
revol_util              float32
total_acc               float64
initial_list_status       int64
application_type          int64
acc_open_past_24mths    float64
mort_acc                float64
pub_rec_bankruptcies    float64
tax_liens               float64
disbursement_method       int64
issue_month              object
issue_year                int32
earliest_cr_year          int32
loan_status               int64
dtype: object

In [6]:
pd.set_option('display.max_columns', 500)
df

,term,int_rate,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,acc_open_past_24mths,mort_acc,pub_rec_bankruptcies,tax_liens,disbursement_method,issue_month,issue_year,earliest_cr_year,loan_status
0,36,7.35,A4,6,OWN,153000.0,1,credit_card,CA,16.99,0.0,Jul-1994,720.0,20.0,1.0,8550.0,22.700001,46.0,1,0,8.0,2.0,1.0,0.0,0,Jun,2017,1994,0
1,36,24.85,E3,2,RENT,50000.0,0,debt_consolidation,WA,6.07,0.0,Jan-2000,685.0,4.0,0.0,3560.0,98.900002,5.0,1,0,2.0,0.0,0.0,0.0,0,Jun,2017,2000,0
2,36,7.35,A4,7,MORTGAGE,110000.0,1,debt_consolidation,TX,13.12,0.0,Mar-2013,710.0,19.0,0.0,23348.0,27.200001,27.0,1,0,13.0,4.0,0.0,0.0,0,Jun,2017,2013,0
3,60,16.02,C5,7,RENT,51979.0,0,debt_consolidation,CA,10.11,0.0,Aug-2006,690.0,15.0,2.0,5733.0,20.000000,21.0,1,0,7.0,0.0,2.0,0.0,0,Jun,2017,2006,0
4,36,16.02,C5,7,MORTGAGE,75000.0,0,debt_consolidation,MA,10.86,1.0,May-2008,685.0,4.0,0.0,2700.0,90.000000,10.0,1,0,5.0,1.0,0.0,0.0,0,Jun,2017,2008,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442330,36,15.59,D1,10,RENT,25400.0,0,credit_card,TX,27.08,0.0,Jul-1999,680.0,9.0,0.0,18266.0,65.000000,18.0,0,0,1.0,0.0,0.0,0.0,0,Jan,2015,1999,0
442331,36,11.99,B5,1,MORTGAGE,63000.0,0,credit_card,OH,23.69,1.0,Jun-1995,685.0,13.0,0.0,21110.0,69.699997,61.0,0,0,11.0,1.0,0.0,0.0,0,Jan,2015,1995,1
442332,60,15.99,D2,5,RENT,35000.0,0,debt_consolidation,TN,30.90,0.0,Sep-2003,680.0,9.0,1.0,11031.0,61.299999,22.0,1,0,7.0,0.0,1.0,0.0,0,Jan,2015,2003,1
442333,60,19.99,E3,1,RENT,64400.0,0,debt_consolidation,MA,27.19,1.0,Oct-2003,695.0,17.0,0.0,8254.0,30.600000,20.0,1,0,6.0,0.0,0.0,0.0,0,Jan,2015,2003,1


In [7]:
df.nunique()

term                        2
int_rate                  200
sub_grade                  35
emp_length                 11
home_ownership              4
annual_inc              28492
verification_status         2
purpose                    14
addr_state                 51
dti                      4206
delinq_2yrs                27
earliest_cr_line          688
fico_range_low             38
open_acc                   67
pub_rec                    26
revol_bal               59308
revol_util               1209
total_acc                 127
initial_list_status         2
application_type            2
acc_open_past_24mths       48
mort_acc                   35
pub_rec_bankruptcies       11
tax_liens                  25
disbursement_method         2
issue_month                12
issue_year                  3
earliest_cr_year           68
loan_status                 2
dtype: int64

In [29]:
field_types=  {
                "term": {
                    "type": "boolean"
                            }, 
                "int_rate": {
                    "type":"numerical",
                    "subtype":"float"
                            },                
                "subgrade":  {
                    "type": "categorical"
                    },                
                "emp_length": {
                    "type" : "numerical",
                    "subtype" : "integer"
                    },                
                "home_ownership":  {
                    "type": "categorical"
                    },                
                "annual_inc": {
                    "type":"numerical",
                    "subtype":"float"
                            },                
                "verification_status": {
                    "type": "boolean"
                            },    
                "purpose":  {
                    "type": "categorical"
                    },                
                "addr_state":  {
                    "type": "categorical"
                    },                
                "dti": {
                    "type":"numerical",
                    "subtype":"float"
                            },                    
                "delinq_2yrs": {
                    "type":"numerical",
                    "subtype":"float"
                            },                    
                "earliest_cr_line":  {
                    "type": "categorical"
                    },                   
                "fico_range_low": {
                    "type":"numerical",
                    "subtype":"float"
                            },                            
                "open_acc": {
                    "type":"numerical",
                    "subtype":"float"
                            },                            
                "pub_rec": {
                    "type":"numerical",
                    "subtype":"float"
                            },                            
                "revol_bal": {
                    "type":"numerical",
                    "subtype":"float"
                            },                            
                "revol_util": {
                    "type":"numerical",
                    "subtype":"float"
                            },                            
                "total_acc": {
                    "type":"numerical",
                    "subtype":"float"
                            },                            
                "initial_list_status": {
                    "type": "boolean"
                            },
                "application_type": {
                    "type": "boolean"
                            },                   
                "acc_open_past_24mths": {
                    "type":"numerical",
                    "subtype":"float"
                            },  
                "mort_acc": {
                    "type":"numerical",
                    "subtype":"float"
                            },  
                "pub_rec_bankruptcies": {
                    "type":"numerical",
                    "subtype":"float"
                            },  
                "tax_liens": {
                    "type":"numerical",
                    "subtype":"float"
                            },  
                "disbursement_method": {
                    "type": "boolean"
                            },   
                "issue_month":  {
                    "type": "categorical"
                    },                         
                "issue_year": {
                    "type" : "numerical",
                    "subtype" : "integer"                
                    },                   
                "earliest_cr_year": {
                    "type" : "numerical",
                    "subtype" : "integer"                
                    },
                "loan_status": {
                    "type": "boolean"
                            }
}


In [28]:
dict_col_names_type = dict(zip(df.dtypes.index, df.dtypes.map(lambda x : x.name).map({'object': 'categorical', 'float64': 'float','float32': 'float', 'int64': 'integer','int32': 'integer'})))
dict_col_names_type

{'term': 'integer',
 'int_rate': 'float',
 'sub_grade': 'categorical',
 'emp_length': 'integer',
 'home_ownership': 'categorical',
 'annual_inc': 'float',
 'verification_status': 'integer',
 'purpose': 'categorical',
 'addr_state': 'categorical',
 'dti': 'float',
 'delinq_2yrs': 'float',
 'earliest_cr_line': 'categorical',
 'fico_range_low': 'float',
 'open_acc': 'float',
 'pub_rec': 'float',
 'revol_bal': 'float',
 'revol_util': 'float',
 'total_acc': 'float',
 'initial_list_status': 'integer',
 'application_type': 'integer',
 'acc_open_past_24mths': 'float',
 'mort_acc': 'float',
 'pub_rec_bankruptcies': 'float',
 'tax_liens': 'float',
 'disbursement_method': 'integer',
 'issue_month': 'categorical',
 'issue_year': 'integer',
 'earliest_cr_year': 'integer',
 'loan_status': 'integer'}